<a href="https://colab.research.google.com/github/667029/dat158-ML-assignment2/blob/main/testDAT158ML2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Baseline model med RandomForestRegressor

In [12]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [13]:
train_url = "https://raw.githubusercontent.com/667029/dat158-ML-assignment2/refs/heads/main/df_train.csv"
test_url = "https://raw.githubusercontent.com/667029/dat158-ML-assignment2/refs/heads/main/df_test.csv"

train_data = pd.read_csv(train_url)
test_data = pd.read_csv(test_url)

train_data = train_data.drop(columns=['date'])
test_data = test_data.drop(columns=['date'])

In [14]:
#print("Treningsdata:")
#print(train_data.head())
#print("\nTestdata:")
#print(test_data.head())

In [15]:
#print("\nManglende verdier i treningsdata:", train_data.isnull().sum())
#print("Manglende verdier i testdata:", test_data.isnull().sum())

In [16]:
train_data = train_data.fillna(train_data.mean())
test_data = test_data.fillna(test_data.mean())

In [17]:
X_train = train_data.drop('price', axis=1)  # Alle kolonner unntatt 'price'
y_train = train_data['price']               # Målvariabelen 'price'

X_test = test_data.drop('price', axis=1)    # Testsett uten 'price'
y_test = test_data['price']                 # Målvariabelen for test

In [18]:
model = RandomForestRegressor(random_state=42)

In [ ]:
model.fit(X_train, y_train)

In [20]:
y_pred = model.predict(X_test)

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

In [ ]:
print("\nModellevaluering:")
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R²):", r2)

In [ ]:
importances = model.feature_importances_
feature_names = X_train.columns
feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
print("\nFeature Importance:")
print(feature_importances)